# YouTube Data Analysing
**Note:** This is not an official YouTube application, nor am I [@flofriday](https://github.com/flofriday), in any way associated with YouTube or Google.

This tool is free software, under the MIT License. \
[GitHub](https://github.com/flofriday/youtube-data)

In [ ]:
# Import libaries
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import numpy as np
import matplotlib as plt
import pandas as pd
from IPython.display import display, Markdown, Latex

import calendar
import json
import os
#%config HistoryManager.enabled = False
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

# Setup Matplotlib
plt.rcParams['figure.dpi'] = 400

# Set the file paths
watch_history = os.path.join("Takeout", "YouTube and YouTube Music", "history", "watch-history.json")
search_history = os.path.join("Takeout", "YouTube and YouTube Music", "history", "search-history.json")

# Load the files
watch_df = pd.read_json(watch_history)
search_df = pd.read_json(search_history)

# Clean up the watch data
watch_df['channel'] = watch_df['subtitles'].map(lambda x: x[0]['name'] if type(x) is list else 'unknown')
watch_df['time'] =  pd.to_datetime(watch_df['time'], infer_datetime_format=True)
#watch_df.drop(columns=['subtitles', 'details', 'products', 'header'], inplace=True)

# Cleanup the search data
search_df['time'] =  pd.to_datetime(search_df['time'], infer_datetime_format=True)
#search_df.drop(columns=['products', 'header', 'titleUrl'], inplace=True)

# Specify the color
COLOR="#f2AAAA"

## Absolut numbers

In [ ]:
def absolut_numbers(watch, search):
    markdown = f"Videos watched since {watch['time'].min().strftime('%b %d %Y')}: **{len(watch)}**<br>"  \
     f"Searches since {search['time'].min().strftime('%b %d %Y')}: **{len(search)}**<br>"  \
     f"Average videos per day: **{len(watch)/((watch['time'].max()-watch['time'].min()).days):.2f}**<br>"  \
     f"Average searches per day: **{len(search)/((search['time'].max()-search['time'].min()).days):.2f}**"
    display(Markdown(markdown))
    
absolut_numbers(watch_df, search_df)

## Creators you watch

In [ ]:
@interact(watch_df=fixed(watch_df), number=widgets.IntSlider(min = 1, max=50, value=24, description='Limit'))
def creator_plot(watch_df, number):
    df = watch_df['channel'].value_counts()
    df.drop(['unknown'], inplace=True)
    df = df.head(number).sort_values()
    plot = df.plot(kind='barh', color=COLOR, figsize=[6.4, number*0.28])
    plot.set_xlabel("videos watched")
    plot.set_title(f'Top {number} creators')
    

## When you watch YouTube
### Overall

In [ ]:
def timeline_plot(watch_df):
    df = pd.DataFrame(watch_df['time'])
    df.set_index('time', inplace=True)
    df['amount'] = 1
    df = df.resample('W').count()
    plot = df.plot(color=COLOR)
    plot.set_title('Timeline')
    plot.set_ylim(ymin= -0.03 * df['amount'].max())
    plot.set_ylabel("videos watched per week")
    plot.set_xlabel("")
    plot.get_legend().remove()

timeline_plot(watch_df)

In [ ]:
# Per Month
def month_plot(watch_df):
    #TODO: Rewrite this with value count
    df = pd.DataFrame()
    df['time'] = watch_df['time']
    df['amount'] = 1
    df = df.groupby(df['time'].dt.month).sum()
    df.index = df.index.map(lambda x: calendar.month_abbr[x])
    plot = df.plot(kind='bar', color=COLOR)
    plot.set_title('Videos watched per month')
    plot.set_ylabel("")
    plot.set_xlabel("")
    plot.get_legend().remove()

# Per weekday
def week_plot(watch_df):
    #TODO: Rewrite this with value count
    df = pd.DataFrame()
    df['time'] = watch_df['time']
    df['amount'] = 1
    df = df.groupby(df['time'].dt.dayofweek).sum()
    df.index = df.index.map(lambda x: calendar.day_abbr[x])
    plot = df.plot(kind='bar',color=COLOR)
    plot.set_title('Videos watched per weekday')
    plot.set_ylabel("")
    plot.set_xlabel("")
    plot.get_legend().remove()
    
# Per hour
def hour_plot(watch_df):
    #TODO: Rewrite this with value count
    df = pd.DataFrame()
    df['time'] = watch_df['time']
    df['amount'] = 1
    df = df.groupby(df['time'].dt.hour).sum()
    df.index = df.index.map(lambda x: f"{x:02}:00")
    plot = df.plot(kind='bar', color=COLOR)
    plot.set_title('Videos watched per hour')
    plot.set_ylabel("")
    plot.set_xlabel("")
    plot.get_legend().remove()
    
@interact(watch_df=fixed(watch_df), timespan = widgets.Dropdown(options=[('Per hour', 1), ('Per day', 2), ('Per month', 3)], value=1, description='Timespan:', ))
def timespan_plot(watch_df, timespan):
    if timespan == 1: 
        hour_plot(watch_df)
    elif timespan == 2:
        week_plot(watch_df)
    elif timespan == 3:
        month_plot(watch_df)

## Searchterms

In [ ]:
@interact(search_df=fixed(search_df), number=widgets.IntSlider(min = 1, max=50, value=24, description='Limit'))
def word_plot(search_df, number):
    # Create a dataframe where the word column has in every row just a single word
    df = pd.DataFrame()
    df['title'] = search_df['title'].replace(regex=['Searched for'], value='')
    df = pd.DataFrame(df.title.str.split(' ').tolist()).stack()
    df = df.reset_index(0)
    df.columns = ['tmp', 'word']
    df = df.reset_index(0)
    df = pd.DataFrame(df.word)
    df['word'].replace('', np.nan, inplace=True)
    df.dropna(subset=['word'], inplace=True)
    
    # Create the plot
    df = df['word'].value_counts()
    df = df.head(number).sort_values()
    plot = df.plot(kind='barh', figsize=[6.4, number*0.28], color=COLOR)
    plot.set_title(f'Top {number} searchwords')